In [ ]:
import os
if os.name == 'nt':
    os.add_dll_directory( 'c:/QtPlatz/bin' )
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QFileDialog
import sqlite3
import plotly.graph_objects as go

In [ ]:
bins = []
counts = []

def plot_histogram( file ):
    print (file);
    conn = sqlite3.connect( file )
    c = conn.cursor()
    offset = 0
    step = 5
    tof = 22.325e-6
    width=10e-9
    tof_lower = tof - width / 2
    tof_upper = tof + width / 2
    bind = ( offset, step, step, tof_lower, tof_upper )
    
    c.execute( "SELECT -Threshold,COUNT(*) AS COUNTS FROM \
    (SELECT MIN(peak_time),ROUND((peak_intensity+?)/?)*? AS Threshold FROM trigger,peak \
        WHERE id=idTrigger AND peak_time > ? AND peak_time < ? GROUP BY id) \
    GROUP BY Threshold", bind )
    
    result_set = c.fetchall()
    for row in result_set:
        bins.append( row[ 0 ] )
        counts.append( row[ 1 ] )
    
    maxCounts = max( counts )
    maxIndex = counts.index( maxCounts )
    bind = ( offset, step, step, tof_lower, tof_upper, maxCounts / 2 )
    
    c.execute( "SELECT sum(-Threshold*COUNTS)/sum(COUNTS) FROM \
    (SELECT *,COUNT(*) AS COUNTS FROM \
        (SELECT MIN(peak_time),ROUND((peak_intensity+?)/?)*? AS Threshold FROM trigger,peak \
            WHERE id=idTrigger AND peak_time > ? AND peak_time < ? GROUP BY id) \
    GROUP BY Threshold ) WHERE COUNTS > ?", bind )
    
    result_set = c.fetchall()
    for row in result_set:
        avgHeight = row[0]
    print( [ avgHeight, maxCounts, maxIndex ] )
    return ( avgHeight, maxCounts, maxIndex, min( bins ), max( bins ) )

In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    filenames, f = QFileDialog.getOpenFileNames(None
                                                , "Open file for export in python"
                                                , "/data/data/wspc"
                                                , "QtPlatz (*.adfs);;All Files (*)" )

    for file in filenames:
        summary = plot_histogram ( file );
        print ( summary ) # height, maxCounts, maxIndex, xmin, xmax
        
        vline_x = summary[ 0 ]
        hline_y = summary[ 1 ] / 2
        maxCounts = summary[ 1 ]
        xmax = summary[ 4 ]
        print ("vline_x=", vline_x, "hline_y=", hline_y, "xmax=", xmax )
        
        fig = go.Figure(data=go.Scatter(x=bins, y=counts))
        fig.add_shape(
            # Line Horizontal
            type="line"
            , x0=0
            , y0=hline_y
            , x1=xmax
            , y1=hline_y
            , line=dict(
                color="LightSeaGreen"
                , width=2
                , dash="dashdot"
            ),
        )
        fig.add_shape(
            # Line Virtical
            type="line"
            , x0=vline_x
            , y0=0
            , x1=vline_x
            , y1=maxCounts
            , line=dict(
                color="LightSeaGreen"
                , width=2
                , dash="dashdot"
            ),
        )
        fig.show() #write_html('figure.html', auto_open=True)
        